In [1]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException, TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from datetime import datetime
import time
import os
import wget
import json
import csv

In [ ]:
test_pair = []
test_pair.append(('https://timesofindia.indiatimes.com/auto/motorsports/', 'AUTO'))
test_pair.append(('https://timesofindia.indiatimes.com/life-style/health-fitness', 'LIFE STYLE'))
# print("1 passed")
xpaths = [
    "//figcaption",
    "//*[contains(concat(' ', @class, ' '), ' sNF1c ')] | //h5",
    "//*[contains(concat(' ', @class, ' '), ' top-newslist ')]//a",
    "//*[contains(concat(' ', @class, ' '), ' top-newslist ')]//*[contains(concat(' ', @class, ' '), ' w_tle ')]//a",
    "//*[contains(concat(' ', @class, ' '), ' title ')]//span",
    "//*[contains(concat(' ', @class, ' '), ' WavNE ')]",
    "//*[contains(concat(' ', @class, ' '), ' desc ')]//a",
    "//*[contains(concat(' ', @class, ' '), ' leadstorties ')]//span",
    "//*[contains(concat(' ', @class, ' '), ' I4QgS ')]",
    "//*[contains(concat(' ', @class, ' '), ' null ')]//span | //figcaption | //*[contains(concat(' ', @class, ' '), ' linktype1 ')]//span",
    "//*[contains(concat(' ', @class, ' '), ' clearfix ')]//div//div//li//a",
    "//*[contains(concat(' ', @class, ' '), ' news-card-img ')]//*[contains(concat(' ', @class, ' '), ' w_tle ')]//a",
    "//div[(count(preceding-sibling::*) + 1 = 8)]//*[contains(concat(' ', @class, ' '), ' M194D ')]//figcaption | //div[(count(preceding-sibling::*) + 1 = 8)]//*[contains(concat(' ', @class, ' '), ' hoid1 ')]//figcaption | //div[(count(preceding-sibling::*) + 1 = 9)]//figcaption | //div[(count(preceding-sibling::*) + 1 = 6)]//figcaption | //*[contains(concat(' ', @class, ' '), ' zxvyz ')]//figcaption | //*[contains(concat(' ', @class, ' '), ' sNF1c ')] | //div[(count(preceding-sibling::*) + 1 = 7)]//figcaption",
    "//*[@id = 'c_headlines_wdt_1']//*[contains(concat(' ', @class, ' '), ' w_tle ')]//a",
    "//*[@id = 'content']//*[contains(concat(' ', @class, ' '), ' w_tle ')]//a",
    "//*[contains(concat(' ', @class, ' '), ' chng_lfttxt ')]//h3 | //*[contains(concat(' ', @class, ' '), ' mrB20 ')]//h3",
]

# print("2 passed")

all_links = []

for item in filtered_pairs:
    webpage_link = item[0]
    webpage_tag = item[1]
    
    # Navigate to the webpage
    driver.get(webpage_link)
    time.sleep(2)
    
    captions = []  # Reset captions for each webpage
    not_found_count = 0
    
    for xpath in xpaths:
        try:
            # Find elements for the current XPath
            elements = driver.find_elements(By.XPATH, xpath)
            
            # If elements are found, extend the captions list
            if elements:
                captions.extend(elements)
        except Exception as e:
            print(f"Error occurred for XPath {xpath}: {e}")  # Log the error for debugging
    print(len(captions))
    # Process the captions if any are found
    if captions:
        hrefs = []

        for caption in captions:
                try:
                    parent = caption
                    anchor_found = False
                    
                    # Traverse up to 5 levels in the hierarchy to search for the <a> tag
                    for i in range(5):
                        try:
                            # Try finding the <a> tag in the current parent level
                            parent_a_tag = parent.find_element(By.XPATH, "./parent::a")
                            
                            # If found, extract the href and break the loop
                            href = parent_a_tag.get_attribute("href")
                            if href:
                                hrefs.append(href)
                                print(f"Current href count: {len(hrefs)}")
                                anchor_found = True
                            # print("pass")
                        except:
                            # Move to the next level in the hierarchy
                            parent = parent.find_element(By.XPATH, "./parent::*")
                    
                    # Move to the next caption if anchor is not found
                    if not anchor_found:
                        not_found_count += 1

                except StaleElementReferenceException:
                    print(f"StaleElementReferenceException encountered. Skipping this caption.")
                    break  # Exit loop after max retries
                
                except Exception as e:
                    # Handle other unexpected exceptions
                    print(f"Error processing caption: {caption.text} - {e}")

        # Create pairs of webpage_tag and hrefs, and add them to the all_links list
        for href in hrefs:
            # print(f"{href}")
            all_links.append((webpage_tag, href))

    else:
        print(f"No captions found for {webpage_tag} ({webpage_link})")

    print(f"Count of 'No <a> tag found within 5 levels' for category {webpage_tag}: {not_found_count}")

all_links = list(set(all_links))
print(len(all_links))
for pair in all_links:
    print(f"Category: {pair[0]}, Link: {pair[1]}")